**SME0213 Otimização Inteira - ICMC-USP**

Amanda Araujo Silva - 10260441

In [1]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 33.1 MB/s eta 0:00:00


In [4]:
import gurobipy as gp

# Atividade Complementar 1: Morango do Amor



In [23]:
from gurobipy import Model, GRB

# Criação do Modelo
model = Model("morango")

# Parêmetros
n = 3  # número de componentes (tipos de morango: T, B, M)
M = 30 # Big M: relação x e y (suficientemente grande dado o problema)

## Dados do problema
cprep = [3.0, 2.5, 3.5] # Custos de preparação (fixo)
cprod = [1.0, 1.5, 1.8] # Custos de produção porção
tprep = [15, 15, 18]    # Tempo de preparação (fixo) (minutos)
tprod = [1.8, 1.8, 2.3] # Tempo de produção (minutos)
valor = [5.0, 4.5, 7.0] # Valor de venda
tmax = 120.0 # limitante da produção (2h de trabalho)

# Adição de variáveis
x = model.addVars(n, vtype=GRB.INTEGER, name="x", lb=0) # inteira não negativa Z+
y = model.addVars(n, vtype=GRB.BINARY, name="y")        # binária

# Definição função objetivo: maximizar lucro (vendas - custos)
model.setObjective(
    sum(valor[i] * x[i] - (cprod[i] * x[i] + cprep[i] * y[i]) for i in range(n)),
    GRB.MAXIMIZE
)

# Restrições
## Tempo
model.addConstr(sum(tprod[i] * x[i] + tprep[i] * y[i] for i in range(n)) <= tmax)

## Big M: relação x e y
for i in range(n):
  model.addConstr(x[i] <= M * y[i])

## Restrição x >= 0 já add pelo uso de lb (lower bound) ao definir x
## Restrição y binário já add ao definir y

# Otimização
model.optimize()


# Escrita do modelo em formato LP
model.write('morango.lp')

# Salva a solução do problema
model.write('morango.sol')

# Exibe os resultados
if model.status == GRB.OPTIMAL:
    for i in range(n):
        print(f"x[{i}] = {x[i].X}")
        print(f"y[{i}] = {y[i].X}")
    print(f"z = {model.ObjVal}")

    # Tempo total de trabalho
    ttrab = sum(tprod[i] * x[i].X + tprep[i] * y[i].X for i in range(n))
    print("Tempo de trabalho (minutos): ", ttrab)

else:
    print("Solução ótima não encontrada.")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 4 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x5234d211
Variable types: 0 continuous, 6 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 144.5000000
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)
Found heuristic solution: objective 174.5000000

Root relaxation: objective 2.084565e+02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [24]:
demanda = [30, 30, 30]

tdemanda = sum(tprod[i] * demanda[i] + tprep[i] for i in range(n))
print("Tempo total demanda (minutos): ", tdemanda)

Tempo total demanda (minutos):  225.0
